# Crawler Transfermarkt

# Social analysis on the transfer market in the Swiss Football League

This Jupyter notebook was created as a result of an assignment in the course "Analysis and Modelling of Social Interactions". Using the newly learned skills in networking techniques, we tried to answer the following research question.

*Research Question:* 

**"Based on the  assumption that Players transfer to regions with similar Language, can we identify contradicting Patterns in the Swiss Football-League?"**

**Module:**   Analysis and Modelling of Social Interactions | W.MSCIDS_AMS03.H2001 <br>
**Authors:**    Gino Cattelan, Konstantinos Lessis, Nico Wyss <br>
**Lecturers:** Isabel Raabe, Nicolas Perony <br>
**Semester:**  HS20  <br>

Date:   18. December 2020

### Load Webpage

Create Variable that tells Transfermarkt that we are a Browser accessing the HTML Code

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import sys
import random
import time
import csv




In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [3]:

#Page for all the relevant football clubs on transfermarkt.de
#Relevant = Played in the first league of Switzerland from 11/12 to 20/21
pages = ['https://www.transfermarkt.de/fc-basel-1893/alletransfers/verein/26','https://www.transfermarkt.de/fc-zurich/alletransfers/verein/260','https://www.transfermarkt.de/bsc-young-boys/alletransfers/verein/452','https://www.transfermarkt.de/fc-sion/alletransfers/verein/321','https://www.transfermarkt.de/fc-thun/alletransfers/verein/938','https://www.transfermarkt.de/fc-luzern/alletransfers/verein/434','https://www.transfermarkt.de/grasshopper-club-zurich/alletransfers/verein/504','https://www.transfermarkt.de/neuchatel-xamax-fcs/alletransfers/verein/9084','https://www.transfermarkt.de/ac-bellinzona/alletransfers/verein/2047','https://www.transfermarkt.de/fc-st-gallen/alletransfers/verein/257','https://www.transfermarkt.de/fc-lausanne-sport/alletransfers/verein/527','https://www.transfermarkt.de/servette-fc/alletransfers/verein/61','https://www.transfermarkt.de/fc-aarau/alletransfers/verein/116','https://www.transfermarkt.de/fc-vaduz/alletransfers/verein/163','https://www.transfermarkt.de/fc-lugano/alletransfers/verein/2790']


In [4]:

#Some teams have more empty tables in the beginning, their links are added to a separate list 
table_exceptions = ["https://www.transfermarkt.de/fc-sion/alletransfers/verein/321",'https://www.transfermarkt.de/grasshopper-club-zurich/alletransfers/verein/504','https://www.transfermarkt.de/servette-fc/alletransfers/verein/61']

### Locate Data and Extract

In [5]:
#will create dataframe for each subtable in order to combine at the end, list to catch subtables
df_collection = []

for page in pages:
    pageTree = requests.get(page, headers=headers)
    soup = BeautifulSoup(pageTree.content, 'html.parser')
    #find name of the team
    team = soup.find('div', id ="verein_head").find('span').string
    #first three tables do not have any content for most of the teams
    if page not in table_exceptions:
        tables = soup.find_all('div','large-6 columns')[3:]
    #some teams have more tables in the beginning without any content, they have been added to a different list of urls
    else:
        tables = soup.find_all('div','large-6 columns')[5:]
    #set how many seasons we want to track
    seasons_counter = 0
    #for every subtable get information needed and put into dictionary
    for table in tables:
        if seasons_counter < 20:
            seasons_counter += 1
            dict = {}
            #titel of every subtable
            dict["Season"] = table.find("h2").string.strip()
            #every tr tag contains all information of every player
            players = table.tbody.find_all('tr')
            player_names = []
            player_clubs = []
            player_transfervalues = []
            for player in players:
                #get the name of the player
                player_names.append(player.find('td','hauptlink').a.string)
                #get the club of the player
                player_clubs.append(player.find('td','no-border-links').a.string)
                #get the transfervalues
                player_transfervalues.append(player.find('td','rechts').string)
            
            #add list of player name to dictionary
            dict["Name"]= player_names
            #add list of transfervalues to dictionary
            dict["Transfer Sum"]= player_transfervalues

            #if the the subtable shows arrivals, add club name to "from" column
            if "Zugänge" in dict["Season"]:
                dict["From"] = player_clubs
                #filling in "to" column with team that we are crawling, making sure columns have same length
                dict["To"] = [team for x in range(len(player_names))]
            #if the subtable shows departures, add club name to "to" table
            else:
                #filling in "from" column with team that we are crawling, making sure columns have same length
                dict["From"] = [team for x in range(len(player_names))]
                dict["To"] = player_clubs
        
            #make season value list as long as player_names list
  
            dict["Season"] = [dict["Season"] for x in range(len(player_names))]
            #print(dict)
            #create data frame
            df = pd.DataFrame.from_dict(dict)
            print(df)
            #add to data frame collection
            df_collection.append(df)

        
#combine data frames
result = pd.concat(df_collection,ignore_index = True)
result.to_csv("all")
print("the file has been created")



           Season               Name Transfer Sum             From  \
0   Zugänge 20/21      Arthur Cabral  4,40 Mio. €        Palmeiras   
1   Zugänge 20/21          Kaly Sene  4,00 Mio. €     Juventus U19   
2   Zugänge 20/21      Edon Zhegrova  3,00 Mio. €        KRC Genk    
3   Zugänge 20/21      Andrea Padula   450 Tsd. €      FC Wil 1900   
4   Zugänge 20/21      Heinz Lindner   ablösefrei  Wehen Wiesbaden   
5   Zugänge 20/21      Pajtim Kasami   ablösefrei          FC Sion   
6   Zugänge 20/21              Jorge        Leihe        AS Monaco   
7   Zugänge 20/21         Timm Klose        Leihe     Norwich City   
8   Zugänge 20/21        Lirik Vishi            -     FC Basel U21   
9   Zugänge 20/21     Felix Gebhardt            -     FC Basel U18   
10  Zugänge 20/21  Liam Chipperfield            -     FC Basel U18   

               To  
0   FC Basel 1893  
1   FC Basel 1893  
2   FC Basel 1893  
3   FC Basel 1893  
4   FC Basel 1893  
5   FC Basel 1893  
6   FC Basel 1893  

### Scrape Country Information

In [7]:
#import google package to search for teams and their countries
#check out https://www.geeksforgeeks.org/performing-google-search-using-python-code/
#Install google package to scrape countries
import sys
!{sys.executable} -m pip install google
#Import search
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found")

In [10]:
#import dataframe
input_location = os.getcwd()
all = pd.read_csv(input_location + "/all")

#Get all unique values from the "From" and "To" column
all_teams_from = all["From"].tolist()


all_teams_to = all["To"].tolist()

#combine the lists
all_teams_unique = list(set(all_teams_from + all_teams_to))


In [11]:
dict_countries = {}

In [15]:
#

headers2_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36']

#dictionary that will contain the result: {'Boavista Porto': 'Portugal', 'Sliema Wander.': 'Malta', 'Thun U21': 'Switzerland'
           
def country_crawler():
    global dict_countries
    try:
        for team in all_teams_unique[len(dict_countries):]:
            query = "transfermarkt club profile "+ team
            for j in search(query, tld="co.in", num=1, stop=1, pause=60):
                headers2_agent = random.choice(headers2_list)
                headers2 = {'User-Agent': headers2_agent}
                pageTree = requests.get(j, headers=headers2)
                soup_country = BeautifulSoup(pageTree.content, 'html.parser')
            try:
                country = soup_country.find('div', id ="verein_head").find('span','mediumpunkt').find('img')["title"]
                dict_countries[team] = country
            except Exception:
                dict_countries[team] = 'Not Found'
            print(len(dict_countries))
                    
    except Exception:
        print("trying to work around connection error")
        time.sleep(120)
        country_crawler()
        
#resulting dict_countries output was copy pasted to file in "01_input" folder, filename: "backup_dict_countries_904.rtf1"
        
        

In [ ]:
#Running Function to get country results, takes 20+ hours, no need to execute, final dictionary and file already created
country_crawler()

In [ ]:
#Hardcoded dictionary to not have to run country_crawler() when whole notebook is executed
dict_countries = {'FC Prilly': 'Not Found', 'Bari': 'Italy', 'Hoffenheim II': 'Germany', 'Dessel Sport': 'Belgium', 'Renate': 'Italy', 'Zenit II': 'Russia', 'Táchira': 'Venezuela', 'Parma Calcio': 'Italy', 'SV Darmstadt 98': 'Germany', 'APOEL Nikosia': 'Cyprus', 'AC Le Havre': 'France', 'Osmanlispor': 'Turkey', 'RC Lens': 'France', 'Standard Res.': 'Not Found', 'Anzio': 'Not Found', 'Djurgarden': 'Sweden', 'NK Osijek': 'Croatia', 'Kansas City': 'United States', 'H. Beer Sheva': 'Israel', 'Ekenäs IF': 'Finland', 'Castellanzese': 'Not Found', 'Olymp. Lyon B': 'France', 'Athletic Adjamé': 'Not Found', 'KS Samara': 'Russia', 'Aargau U21': 'Not Found', 'Tochigi SC': 'Japan', 'FC Aarau': 'Switzerland', 'UD Levante': 'Spain', 'FC Cadenazzo': 'Not Found', 'Alashkert': 'Armenia', 'West Ham U23': 'England', 'FC Augsburg': 'Germany', 'Esperance': 'Tunisia', 'Legia Warschau': 'Poland', 'FC Naters': 'Switzerland', 'Grasshopper Club Zürich': 'Switzerland', 'BSC YB U21': 'Switzerland', 'Ponferradina': 'Spain', 'Charlotte Ind.': 'United States', 'Pro Vercelli': 'Italy', 'FC Sochaux': 'France', 'Barracas C.': 'Argentina', 'Haladás': 'Hungary', 'VfL Osnabrück': 'Germany', 'NK Zagreb': 'Not Found', 'Hansa Rostock': 'Germany', 'D. Concepción': 'Not Found', 'FC Portalban': 'Switzerland', 'Hamrun Spart.': 'Malta', 'Clodiense': 'Italy', 'K. Offenbach': 'Germany', 'ES Troyes AC': 'France', 'FC Southampton': 'England', 'JX Liansheng': 'China', 'Caxias-RS': 'Not Found', 'Yverdon-Sports FC II (U21)': 'Switzerland', 'GC Biaschesi': 'Not Found', 'Al-Jazira': 'United Arab Emirates', 'APO Levadiakos': 'Greece', 'Ascoli': 'Italy', 'Bröndby IF': 'Denmark', 'Bayern München': 'Germany', 'Roter Stern': 'Not Found', 'SC Kriens': 'Switzerland', 'Suwon Bluewings': 'Korea, South', 'Chicago Fire': 'United States', 'Strømsgodset': 'Norway', 'Zenit S-Pb': 'Russia', 'AOK Kerkyra': 'Not Found', 'Schwamendingen': 'Not Found', 'FK Rostov': 'Russia', 'SC Freiburg': 'Germany', 'Orlando City SC': 'United States', 'FC Fulham': 'England', 'FC Utrecht': 'Netherlands', 'Fréjus-St-Raph.': 'France', 'SC Cambuur': 'Netherlands', 'Central Coast': 'Australia', 'AS Livorno': 'Italy', 'Red Star FC': 'France', 'Herediano': 'Costa Rica', 'Udinese Calcio': 'Italy', 'Wolves U23': 'England', 'Zeljeznicar': 'Bosnia-Herzegovina', 'Chur 97': 'Switzerland', 'Shimizu S-Pulse': 'Japan', 'Genua CFC': 'Italy', 'Brescia': 'Italy', 'Kayserispor': 'Turkey', 'FC Heartland': 'Not Found', 'FC Schalke 04': 'Germany', 'FSV Bissingen': 'Germany', 'Fulham U23': 'England', 'Ural': 'Russia', 'C. Rizespor': 'Turkey', 'Bnei Yehuda': 'Israel', 'SC Cham': 'Switzerland', 'Losone': 'Not Found', 'AC Bellinzona': 'Switzerland', 'Newcastle Utd.': 'England', 'Glasgow Rangers': 'Scotland', 'WAFA SC': 'Ghana', 'Sandnes Ulf': 'Norway', 'OC Blues FC': 'United States', 'Pergolettese': 'Italy', 'Cerezo Osaka': 'Japan', 'Erzgebirge Aue': 'Germany', 'Samsunspor': 'Turkey', 'Smouha': 'Egypt', 'FC Groningen': 'Netherlands', 'RFS': 'Latvia', 'Stade Rennes': 'France', 'Juventus U23': 'Italy', 'FC Girona': 'Spain', 'FK Tuzla City': 'Bosnia-Herzegovina', 'Latina Calcio': 'Italy', 'Guarani': 'Paraguay', "1.FC K'lautern": 'Germany', 'Ludogorets': 'Bulgaria', 'Tombense': 'Brazil', 'US Giubiasco': 'Not Found', 'Palmeiras': 'Brazil', 'Buriram Utd.': 'Thailand', 'Dundee United': 'Scotland', 'Montpellier': 'France', 'Servette U18': 'Switzerland', 'AIK Solna': 'Sweden', 'Hajduk Split': 'Croatia', 'Union Ripa': 'Not Found', 'Kaposvár': 'Hungary', 'Bournemouth': 'England', 'KSC Lokeren': 'Not Found', 'Al-Gharafa SC': 'Qatar', 'FK Sarajevo': 'Bosnia-Herzegovina', 'FC Breitenrain': 'Switzerland', 'Berliner AK': 'Germany', 'Dunajska Streda': 'Slovakia', 'Wigan U23': 'Not Found', 'FC Ingolstadt': 'Germany', 'Holstein Kiel': 'Germany', 'Wigan Athletic': 'England', 'OGC Nizza': 'France', 'AC Mailand U19': 'Not Found', 'FC Littau': 'Not Found', 'Zamalek': 'Egypt', 'SCO Angers B': 'France', 'RC Grasse': 'France', 'San Lorenzo II': 'Not Found', 'Le Mont LS': 'Not Found', 'Septemvri Sofia': 'Bulgaria', 'Hapoel Akko': 'Israel', 'ZSKA Sofia': 'Bulgaria', 'AS Nancy U19': 'Not Found', 'GC Zürich U21': 'Switzerland', 'Flamengo': 'Brazil', 'FC Basel 1893': 'Switzerland', 'Paris FC': 'France', 'FC Brügge': 'Belgium', 'Kyoto Sanga': 'Japan', 'AS Tanda': 'Not Found', 'Shonan Bellmare': 'Japan', 'FC Monthey': 'Switzerland', 'FC Nantes': 'France', '1.FC Köln': 'Germany', 'Lechia Gdansk': 'Poland', 'FC Elche': 'Spain', 'Casale': 'Italy', 'Wrexham': 'England', 'FC Liverpool': 'England', 'Fortaleza EC B': 'Brazil', 'Coritiba FC': 'Brazil', 'Nest-Sotra IL': 'Not Found', 'Inter Mailand': 'Italy', 'Luzern U21': 'Switzerland', 'Racing Club': 'Not Found', 'FC Wohlen U23': 'Not Found', 'Mosta FC': 'Malta', 'Port Vale': 'England', 'Brisbane Roar': 'Australia', 'MSV Duisburg II': 'Germany', 'FC Porto U19': 'Not Found', 'FK Krasnodar': 'Russia', 'FC Grenchen': 'Switzerland', 'FC Porto B': 'Portugal', 'MFK Karvina': 'Czech Republic', 'Malmö FF': 'Sweden', 'Botafogo': 'Brazil', 'D.C. United': 'United States', 'Xamax FCS U21': 'Switzerland', 'Bellinzona Jgd.': 'Switzerland', 'Ankaragücü': 'Turkey', 'FC Koper': 'Slovenia', 'Rappi Jugend': 'Not Found', 'GIF Sundsvall': 'Sweden', 'Stade Tunisien': 'Tunisia', 'Slovan Liberec': 'Czech Republic', 'Esbjerg fB': 'Denmark', 'FC Crotone': 'Italy', 'FC Basel': 'Switzerland', 'Martigny-Sports': 'Switzerland', 'Rennes B': 'France', 'Wisla Krakau': 'Poland', 'Esteghlal FC ': 'Iran', 'FC Riga': 'Latvia', 'Zob Ahan': 'Iran', 'FC Getafe': 'Spain', 'Júbilo Iwata': 'Japan', 'Würzb. Kickers': 'Germany', 'Ross County': 'Scotland', 'Santo André-SP': 'Brazil', 'FSV Frankfurt': 'Germany', 'FC Lorient': 'France', 'Sparta Prag': 'Czech Republic', 'FC Winterthur': 'Switzerland', 'SCR Altach': 'Austria', 'HNK Rijeka': 'Croatia', 'HN Jianye': 'China', 'Benfica U23': 'Portugal', 'FC Nitra': 'Slovakia', 'São Bento (SP)': 'Brazil', 'FC Voluntari': 'Romania', 'FC Aarau II': 'Switzerland', 'Molde': 'Norway', 'AS Minusio': 'Not Found', 'FC Cincinnati': 'United States', 'LA Galaxy': 'United States', 'Sloboda Tuzla': 'Bosnia-Herzegovina', 'Danubio FC': 'Uruguay', 'Anzhi': 'Russia', 'FC Haka': 'Finland', 'AS Novazzano': 'Switzerland', 'Göztepe': 'Turkey', 'Preußen Münster': 'Germany', 'US Orléans': 'France', 'E. Frankfurt': 'Germany', 'KAC Kénitra': 'Not Found', 'Real Valladolid': 'Spain', 'USM Algier': 'Algeria', 'Terek Grozny': 'Russia', 'Sanliurfaspor ': 'Turkey', 'Empoli U19': 'Italy', 'AJ Auxerre': 'France', 'KF Tirana': 'Albania', 'FC Wettswil': 'Switzerland', 'CRB': 'Brazil', 'Dalkurd FF': 'Sweden', 'LB Châteauroux': 'France', 'Boavista Porto': 'Portugal', 'Sloboda Uzice': 'Not Found', 'Debrecen': 'Hungary', 'Puskás AFC': 'Hungary', 'FC Sursee': 'Switzerland', 'FC Chelsea': 'England', 'AC Sementina': 'Not Found', 'AO Xanthi': 'Greece', 'Etoile Carouge': 'Switzerland', 'Sepsi OSK': 'Romania', 'BSC Young Boys': 'Switzerland', 'CA Unión': 'Not Found', 'AC Bellinzona U18': 'Not Found', 'NT Zhiyun': 'China', 'Le Mans FC': 'France', 'Dubai CSC': 'Not Found', 'FC Lugano': 'Switzerland', 'Calcio Como': 'Italy', 'Iskra-Stali': 'Not Found', 'Hellas Verona': 'Italy', 'AC Ravecchia': 'Not Found', 'CD Toledo': 'Not Found', 'AZ Alkmaar': 'Netherlands', 'FC Emmen': 'Netherlands', 'Bassano': 'Not Found', 'UD San Fernando': 'Not Found', 'Milano City': 'Not Found', 'Dijon': 'France', 'FC Prishtina': 'Kosovo', 'Leeds United': 'England', 'Stoke U23': 'England', 'Zaglebie Lubin': 'Poland', 'FC Ashdod': 'Israel', 'KRC Genk ': 'Belgium', 'FC Besa': 'Not Found', 'Akad. Pandev': 'North Macedonia', 'SC YF Juventus': 'Switzerland', 'R. B. Linense': 'Spain', 'St. Gallen U21': 'Switzerland', 'Olymp. Genève': 'Switzerland', 'FC Schötz': 'Switzerland', 'Sporting Gijón': 'Spain', 'Livingston FC': 'Scotland', 'KF Feronikeli': 'Kosovo', 'Varese': 'Not Found', 'EN Paralimniou': 'Cyprus', 'Chaux-de-Fonds': 'Switzerland', 'FCI': 'Estonia', 'Independente-SP': 'Not Found', 'Desportivo Aves': 'Not Found', 'Dinamo Minsk': 'Belarus', 'Villa Nova AC': 'Not Found', 'SSC Bari': 'Italy', 'Ceres-Negros FC': 'Philippines', 'VfB Stuttgart': 'Germany', 'BJ Guoan': 'China', 'Sonnenhof-Gr.': 'Germany', 'Granada B': 'Spain', 'Vålerenga': 'Norway', 'NK Domzale': 'Slovenia', 'Olymp. Lyon U19': 'Not Found', 'FC Reading': 'England', 'FC Porza': 'Not Found', 'FC Wangen b. O.': 'Not Found', 'AC Pisa': 'Italy', 'USC Paloma': 'Germany', 'TP Mazembe': 'Not Found', 'BSC Old Boys': 'Switzerland', 'Dynamo Kiew': 'Ukraine', 'FC Dornbirn': 'Austria', 'Lausanne-Sport': 'Switzerland', 'BSK Borca': 'Not Found', 'Gaziantep BB': 'Turkey', 'Quakes': 'United States', 'ASC Gordola': 'Not Found', 'Manisaspor': 'Turkey', 'KAS Eupen': 'Belgium', 'Nea Salamis': 'Cyprus', 'Celik Zenica': 'Not Found', 'FC Weesen': 'Switzerland', 'AC Perugia': 'Italy', 'Wolfsberger AC': 'Austria', 'KFC Uerdingen': 'Germany', 'Hamburger SV': 'Germany', 'AS Dakar Sacré Coeur': 'Not Found', 'Ferencváros': 'Hungary', 'Halmstad': 'Sweden', 'Excelsior': 'Netherlands', 'West Ham Utd.': 'England', 'Sambenedettese': 'Italy', 'Sydney FC': 'Australia', 'F. Düsseldorf': 'Germany', 'Pazova': 'Not Found', 'Monte Azul-SP': 'Not Found', 'Calcio Padova': 'Italy', 'Pr. Niederkorn': 'Luxembourg', 'Sheriff': 'Moldova', 'Sopot': 'Not Found', 'FC Córdoba': 'Spain', 'Beerschot V.A.': 'Belgium', 'Colorado Rapids': 'United States', 'U. de Chile': 'Chile', 'Roda JC': 'Netherlands', 'FC Sion U21': 'Switzerland', 'Bnei Sachnin': 'Israel', 'Gimnàstic': 'Spain', 'Ararat Erewan': 'Armenia', 'FC Ebikon': 'Not Found', 'FC Locarno': 'Not Found', 'FC Chiasso': 'Switzerland', 'FK Partizani': 'Albania', 'FC Getafe B': 'Spain', 'FC Dordrecht': 'Netherlands', 'De Graafschap': 'Netherlands', '1º de Agosto': 'Not Found', 'Formentera': 'Not Found', 'FC Basel U18': 'Switzerland', 'Ternana': 'Italy', 'FC Bologna': 'Italy', 'FC Kreuzlingen': 'Switzerland', 'AS Pikine': 'Not Found', 'Liverpool FC': 'England', 'Rayo Vallecano': 'Spain', 'M. Tel Aviv': 'Israel', 'Tromsø IL': 'Norway','RB Leipzig': 'Germany', 'FC Astana': 'Kazakhstan', 'Hamilton Acad.': 'Scotland', 'Hammarby': 'Sweden', 'Werder Bremen': 'Germany', 'FC Emmenbrücke': 'Switzerland', 'SC Zofingen': 'Switzerland', 'Cremonese': 'Italy', 'Mirassol-SP': 'Brazil', 'St. Gallen U18': 'Switzerland', 'Aargau U18': 'Switzerland', "Bor. M'gladbach": 'Germany', 'FC Lausanne-Sport': 'Switzerland', 'BB Erzurumspor': 'Turkey', 'KF Teuta': 'Albania', 'Tricordiano-MG': 'Not Found', 'Figueirense FC': 'Brazil', 'R. Majadahonda': 'Spain', 'La Vitréenne FC': 'Not Found', 'Syrianska FC': 'Not Found', 'FC Dallast': 'United States', 'Defensor': 'Uruguay', 'FC Arsenal': 'England', 'Inter U19': 'Not Found', "Newell's II": 'Not Found', 'Rac. Santander': 'Spain', 'IFK Mariehamn': 'Finland', 'FC Santos': 'Brazil', 'Grenoble': 'France', 'Stade-Lausanne': 'Switzerland', 'AS Monaco B': 'France', 'Bradford City': 'England', 'Carpi': 'Italy', 'Grasshoppers': 'Switzerland', 'FC Eindhoven': 'Netherlands', 'FC Wohlen': 'Switzerland', 'Rapid Bukarest': 'Romania', 'Jagodina': 'Serbia', 'AC Florenz': 'Italy', 'PFC Beroe ': 'Bulgaria', 'FC Vaduz U18': 'Switzerland', 'Amiens SC': 'France', 'Honvéd': 'Hungary', 'Antalyaspor': 'Turkey', 'Coritiba FC U20': 'Not Found', 'Feyenoord': 'Netherlands', 'CA Bastia': 'Not Found', 'Zulte Waregem': 'Belgium', 'FC Fribourg': 'Switzerland', 'Estoril Praia': 'Portugal', 'Athletico-PR': 'Brazil', 'Vevey United': 'Switzerland', 'US Arbedo': 'Not Found', 'B. Dortmund U19': 'Germany', 'Club Africain': 'Tunisia', 'Górnik Zabrze': 'Poland', 'FC Montréal': 'United States', 'FC Vaduz': 'Switzerland', 'Real Saragossa': 'Spain', 'Stallion FC': 'Philippines', 'Huachipato FC': 'Chile', 'Pianese': 'Italy', 'Schweinfurt 05': 'Germany', 'AS Monaco': 'France', 'River Plate': 'Not Found', 'Beira-Mar': 'Not Found', 'FC Tuggen': 'Switzerland', 'FC Colombier': 'Switzerland', 'Wadi Degla ': 'Egypt', 'Olginatese': 'Not Found', 'Austria Wien': 'Austria', 'Viborg FF': 'Denmark', 'Sporting': 'Portugal', 'MZ Hakka': 'China', 'FC Gambarogno': 'Switzerland', 'Slaven Belupo': 'Croatia', 'Catania Calcio': 'Italy', 'TZ Yuanda': 'China', 'SC Heerenveen': 'Netherlands', 'Umm Salal SC': 'Qatar', 'FC St. Gallen 1879': 'Switzerland', 'Norwich City': 'England', 'Karriereende': 'Not Found', 'Odense BK': 'Denmark', 'Servette U21': 'Switzerland', 'Wolverhampton': 'England', 'Olimpia': 'Paraguay', 'Vit. Guimarães': 'Portugal', 'Sivasspor': 'Turkey', 'Servette U17': 'Switzerland', 'Vaduz II': 'Not Found', 'Philadelphia': 'United States', 'Inter Zapresic': 'Croatia', 'FC Lugano U21': 'Switzerland', 'Lancy FC': 'Switzerland', 'Gil Vicente': 'Portugal', 'HJK Helsinki': 'Finland', 'Aargau U16': 'Not Found', 'Vereinslos': 'Not Found', 'FC Malcantone': 'Not Found', 'FC Fidene': 'Not Found', 'AS Cannes': 'Not Found', 'KF Vllaznia': 'Albania', 'Al-Taawon': 'Saudi Arabia', 'Tubarão-SC': 'Not Found', 'Stade Gabèsien': 'Not Found', 'E. Braunschweig': 'Germany', 'FC Turin': 'Italy', 'GFC Ajaccio': 'France', 'Catanzaro': 'Italy', 'Arsenal U21': 'England', 'FC Tours': 'Not Found', 'Brighton & Hove': 'England', 'CD Leganés': 'Spain', 'FC Solothurn': 'Switzerland', 'Union Berlin': 'Germany', 'FC Thun': 'Switzerland', 'Timbers': 'United States', 'Leixões': 'Portugal', 'BSC YB U18': 'Switzerland', 'NEC Nijmegen': 'Netherlands', 'NY Cosmos': 'Not Found', 'Lazio Rom': 'Italy', 'AC Ajaccio': 'France', 'LOSC Lille': 'France', 'PAS Giannina': 'Greece', 'Macva': 'Serbia', 'Dynamo Dresden': 'Germany', 'Metalac': 'Serbia', 'Stade Brest 29': 'France', 'FC Zürich U21': 'Switzerland', 'CF Pachuca': 'Mexico', "J'ville Armada": 'Not Found', 'Team Vaud U18': 'Switzerland', 'Inverness Cal.': 'Scotland', 'AC Arles': 'Not Found', 'Siroki Brijeg': 'Bosnia-Herzegovina', 'Neftchi Baku': 'Azerbaijan', 'Partizan': 'Serbia', 'RCD Mallorca': 'Spain', 'Admira Wacker': 'Austria', 'AFC Astra': 'Romania', 'NK Zadar ': 'Not Found', 'FC Sion U18': 'Switzerland', 'Rubin Kazan': 'Russia', 'Rapid Lugano': 'Not Found', 'Spartak Moskau': 'Russia', 'Cibalia': 'Croatia', 'Spezia Calcio': 'Italy', 'Panionios': 'Not Found', 'Siena': 'Italy', 'FC Baulmes': 'Not Found', 'Neuweg': 'Not Found', 'M. Petah Tikva': 'Israel', 'Benfica': 'Portugal', 'FC Mendrisio': 'Switzerland', 'Liberty Prof.': 'Ghana', 'Vestsjaelland': 'Not Found', 'Puerto Rico FC': 'Not Found', 'SC Bastia': 'France', 'US Palermo': 'Italy', 'Chelsea U23': 'England', 'Konyaspor': 'Turkey', 'Greuther Fürth': 'Germany', 'VfR Aalen': 'Germany', 'Neapel U19': 'Italy', 'Eastern Sub.': 'New Zealand', 'Wehen Wiesbaden': 'Germany', 'FC City': 'England', 'Vedeggio': 'Not Found', 'Kalmar FF': 'Sweden', 'Petrolul': 'Romania', 'FC Balzers': 'Switzerland', 'AC Taverne': 'Switzerland', 'Barca U19': 'Not Found', 'Hamburger SV II': 'Germany', 'FC Luzern': 'Switzerland', 'VfL Wolfsburg': 'Germany', 'Novara U19': 'Not Found', 'Gaziantepspor': 'Not Found', 'Middlesbrough': 'England', 'WSG Tirol': 'Austria', 'Petro Luanda': 'Not Found', 'Chievo Verona': 'Italy', 'MFM FC': 'Not Found', 'LR Vicenza': 'Italy', 'FC Granada': 'Spain', 'Y. Malatyaspor': 'Turkey', 'Malcantone Agno': 'Not Found', 'AS Cittadella': 'Italy', 'FC Aesch': 'Not Found', 'Vicenza': 'Italy', 'Eskisehirspor': 'Turkey', 'Amarante': 'Not Found', 'Sabah FK': 'Azerbaijan', 'Stade Nyonnais': 'Switzerland', 'El Mokawloon': 'Egypt', 'Goiás EC': 'Brazil', "M'gladbach II": 'Germany', 'Apollon Smyrnis': 'Greece', 'FC Sierre': 'Switzerland', 'AS Rom': 'Italy', 'TSG Hoffenheim': 'Germany', 'FC Köniz': 'Switzerland', 'FC Ascona': 'Not Found', 'Al-Fateh': 'Saudi Arabia', 'KV Kortrijk': 'Belgium', 'SKN St. Pölten': 'Austria', 'Stade Reims': 'France', 'AE Larisa': 'Greece', 'Sampdoria': 'Italy', 'FC Zürich': 'Switzerland', 'FC Donneloye': 'Not Found', 'KAA Gent': 'Belgium', 'OFK Belgrad U19': 'Not Found', 'Atal. Bergamo': 'Italy', 'AJ Auxerre B': 'France', 'FC Alle': 'Not Found', 'GC Zürich U18': 'Switzerland', 'Royal Mouscron': 'Belgium', 'Swansea City': 'England', 'Belenenses SAD': 'Portugal', 'Willem II': 'Netherlands', 'FC Cádiz': 'Spain', 'Sporting U19': 'Not Found', 'ND Gorica': 'Slovenia', 'Anorthosis': 'Cyprus', 'Nîmes Olympique': 'France', 'Spartak Trnava': 'Slovakia', 'Flying Sports': 'Not Found', 'SC Paderborn': 'Germany', 'Irapuato': 'Not Found', 'Western Sydney': 'Australia', 'San Severo': 'Not Found', 'Wolfsburg II': 'Germany', 'Vélez U20': 'Not Found', 'Ethnikos': 'Not Found', 'Miedz Legnica': 'Poland', 'Ajax Amsterdam': 'Netherlands', 'Hertha BSC': 'Germany', 'Rudar Prijedor': 'Bosnia-Herzegovina', 'Umeå FC': 'Sweden', 'FC Dietikon': 'Switzerland', 'Badajoz 1905': 'Spain', 'Toronto FC': 'United States', 'Aalborg BK': 'Denmark', 'Progresso': 'Uruguay', 'AEL Limassol': 'Cyprus', 'Huracán': 'Not Found', 'Tskhinvali': 'Not Found', 'Vila Nova FC': 'Brazil', 'Al-Shorta SC': 'Not Found', 'Sturm Graz': 'Austria', 'Renhe Reserve': 'China', 'Casa Pia': 'Portugal', 'Waasl.-Beveren': 'Belgium', 'Maccabi Netanya': 'Israel', 'FC Thalwil': 'Switzerland', 'Ol. Marseille': 'France', 'ES Sahel': 'Tunisia', 'SC Freiburg II': 'Germany', 'Rabotn. Skopje': 'North Macedonia', 'AS Nancy': 'France', 'Perth Glory': 'Australia', 'Hannover 96': 'Germany', 'Etoile Filante': 'Not Found', 'Zamora': 'Venezuela', 'Stade Laval': 'France', 'SC Ritzing': 'Austria', 'Juventus Turin': 'Italy', 'NK Ankaran': 'Not Found', 'Rampla Jrs': 'Uruguay', 'FC San Pedro': 'Not Found', 'Arsenal FC II': 'England', 'Vitesse Arnheim': 'Netherlands', 'IFK Norrköping': 'Sweden', 'FC Wacker': 'Austria', 'CS Chênois': 'Switzerland', 'Hapoel Raanana': 'Israel', 'Kecskeméti TE': 'Not Found', 'VfB Auerbach': 'Germany', 'Omonia Nikosia': 'Cyprus', 'Dundee FC': 'Scotland', 'Pandurii ': 'Romania', 'AEK Larnaka': 'Cyprus', 'Yverdon-Sport': 'Switzerland', 'FC Sevilla': 'Spain', 'Niarry Tally': 'Not Found', 'Argentinos II': 'Not Found', 'Aktobe': 'Kazakhstan', 'Viktoria Pilsen': 'Czech Republic', 'FC Baden': 'Switzerland', 'AS Kaloum': 'Not Found', 'FC Postojna': 'Not Found', 'FC Linth 04': 'Switzerland', 'Enppi SC': 'Egypt', 'Travnik': 'Bosnia-Herzegovina', 'FC Sion': 'Switzerland', 'FC Valenciennes': 'France', 'Los Angeles FC': 'United States', 'FC Andorra': 'Spain', 'FC Metz': 'France', 'VVV-Venlo': 'Netherlands', 'Enisey': 'Russia', ' AS Castello': 'Not Found', 'Unbekannt': 'Not Found', 'Albacete': 'Spain', 'FC Biel-Bienne': 'Switzerland', '1860 München': 'Germany', 'FC Serrières': 'Not Found', 'FC Drita': 'Kosovo', 'UR La Louvière': 'Belgium', 'SR Delémont': 'Switzerland', 'B. München II': 'Germany', 'Akhisar Belediye': 'Turkey', 'Parma': 'Italy', 'Frosinone Calc.': 'Italy', 'Kawkab Marrakech': 'Not Found', 'NK Aluminij': 'Slovenia', 'Deportivo Lara': 'Venezuela', 'United Zürich': 'Switzerland', 'PAOK Saloniki': 'Greece', '1.FSV Mainz 05': 'Germany', 'EC Bahia': 'Brazil', '1.FC Köln II': 'Germany', 'Peñarol': 'Uruguay', 'Novara Calcio': 'Italy', 'Melbourne Vict.': 'Australia', 'Grosseto': 'Italy', 'ZSKA Moskau': 'Russia', 'RSC Anderlecht': 'Belgium', 'FC Zürich U18': 'Switzerland', 'NE Xamax': 'Switzerland', 'Hapoel Tel Aviv': 'Israel', 'Dep. Alavés': 'Spain', 'FC Porto': 'Portugal', 'Heart of Midl.': 'Scotland', 'Piacenza': 'Italy', 'SCO Angers': 'France', 'JS Suning': 'China', 'A. Lustenau': 'Austria', 'Montreal Impact': 'United States', 'FCC II': 'Not Found', 'Giulianova': 'Italy', 'Twente Enschede': 'Netherlands', 'Arapongas': 'Not Found', 'Arzanese': 'Not Found', 'Thonon Évian': 'Not Found', 'FC Konolfingen': 'Switzerland', 'NAC Breda': 'Netherlands', 'BEJUNE U18': 'Switzerland', 'AC Vallemaggia': 'Not Found', 'Skonto': 'Not Found', 'Yangon Utd.': 'Myanmar', 'Ruch Chorzow': 'Poland', 'UD Logroñés': 'Spain', 'Delfino Pescara': 'Italy', 'Kiryat Shmona': 'Israel', 'FC Diessenhofen': 'Not Found', 'Neuchâtel Xamax FCS': 'Switzerland', 'Meyrin FC': 'Switzerland', 'Ittihad': 'Saudi Arabia', 'Xamax': 'Switzerland', 'Inter Baku': 'Azerbaijan', 'BATE Borisov': 'Belarus', 'Nacional': 'Uruguay', 'São Carlos': 'Not Found', 'FC Corcelles': 'Not Found', 'Lech Posen': 'Poland', 'Sport. Cristal': 'Peru', 'Zorya Lugansk': 'Ukraine', 'FK Indjija': 'Serbia', 'Independiente': 'Not Found', 'Spartaks': 'Russia', 'FC Aigle': 'Not Found', 'Vélez II': 'Not Found', 'Radnik': 'Serbia', 'CS Sfaxien': 'Tunisia', 'FC Valencia': 'Spain', 'SC Braga B': 'Not Found', 'Mersin IY': 'Not Found', 'FC St. Gallen': 'Switzerland', 'Mineros': 'Venezuela', 'Caracas FC': 'Venezuela', 'FC Le Mans B': 'France', 'FC Basel U21': 'Switzerland', 'Ermis Aradippou': 'Cyprus', 'Rimyongsu SC': 'Not Found', 'FC Bassecourt': 'Switzerland', 'Rapperswil-Jona': 'Switzerland', 'Austria Wien II': 'Austria', 'Monza': 'Italy', 'Oldham Athletic': 'England', 'Cercle Brügge': 'Belgium', 'Stuttg. Kickers': 'Germany', 'Chern. Burgas': 'Not Found', 'Boca Juniors': 'Not Found', 'Slask Wroclaw': 'Poland', 'Dinamo Tiflis': 'Georgia', 'FK Ufa': 'Russia', 'FC Schaffhausen': 'Switzerland', 'Betis Sevilla': 'Spain', 'FK Orenburg': 'Russia', 'FC Everton': 'England', 'Xamax Jugend': 'Not Found', 'Urania Genève': 'Switzerland', 'FC Bavois': 'Switzerland', 'Valencia B': 'Spain', 'Sampdoria U19': 'Italy', 'B. Jerusalem': 'Israel', 'União Madeira': 'Not Found', 'SC Brühl SG': 'Switzerland', 'Team Ticino U18': 'Switzerland', 'FC Toulouse': 'France', 'US Sassuolo': 'Italy', 'Arm. Bielefeld': 'Germany', 'SC Balerna': 'Not Found', 'Team Vaud U21': 'Switzerland', 'Argentina': 'Not Found', 'Adelaide United': 'Australia', 'AFC Eskilstuna': 'Sweden', 'Bor. Dortmund': 'Germany', 'Olympique Lyon': 'France', 'FC Empoli': 'Italy', 'CD Tondela': 'Portugal', 'Stade-Payerne': 'Switzerland', 'Saint-Étienne B': 'Not Found', 'Radnicki Nis': 'Serbia', 'Arsenal U23': 'England', 'FC Muri': 'Switzerland', 'CD Guadalajara': 'Mexico', 'FC Black Stars': 'Switzerland', 'SC Buochs': 'Switzerland', 'Chieti': 'Not Found', 'FC Dürrenast': 'Not Found', 'Servette FC': 'Switzerland', 'Sanremese ': 'Italy', 'Juventus U19': 'Not Found', 'Tractor': 'Iran', 'Piast Gliwice': 'Poland', 'MSV Duisburg': 'Germany', 'Rapid Wien': 'Austria', 'AFC Sunderland': 'England', 'Xamax II': 'Not Found', 'Pau FC': 'France', 'US Boulogne': 'France', 'Olymp. Piräus': 'Greece', 'Mladost': 'Not Found', 'Albirex Niigata': 'Japan', 'Acad. Coimbra': 'Portugal', 'Le Locle Sports': 'Switzerland', 'Minnesota Utd.': 'United States', 'Fluminense U20': 'Not Found', 'Eschen-Mauren': 'Switzerland', 'Ruzomberok': 'Slovakia', 'Venezia': 'Italy', 'SC Düdingen': 'Switzerland', 'SV Kapfenberg': 'Austria', 'Atlético-GO': 'Brazil', 'NK Olimpija': 'Slovenia', 'Slavia Prag': 'Czech Republic', 'Barcelona SC': 'Ecuador', 'Sliema Wander.': 'Malta', '1.FC Nürnberg': 'Germany', 'SV Höngg': 'Switzerland', 'Toronto FC II': 'Not Found', 'TSV Hartberg': 'Austria', 'Altay SK': 'Turkey', 'Hallescher FC': 'Germany', 'Stand. Lüttich': 'Belgium', 'OFK Belgrad': 'Not Found', 'Levski Sofia': 'Bulgaria', 'Kasimpasa': 'Turkey', 'FC Echallens': 'Switzerland', 'Karlsruher SC': 'Germany', 'Metropolitano': 'Not Found', 'Nordsjaelland': 'Denmark', 'Vancouver': 'United States', 'Columbus Crew': 'United States', 'Carl Zeiss Jena': 'Germany', 'Fortuna Sittard': 'Netherlands', 'RB Salzburg': 'Austria', 'Dinamo Zagreb': 'Croatia', 'R. Straßburg': 'France', 'Sunshine Stars': 'Not Found', 'Pune City': 'Not Found', 'FC Málaga': 'Spain', 'Fluminense': 'Brazil', 'Aston Villa': 'England', 'Luzern U18': 'Switzerland', 'Napredak': 'Serbia', 'FC Wil 1900': 'Switzerland', 'Revolution': 'United States', 'SV Ried': 'Austria', 'Slo. Bratislava': 'Slovakia', 'Thun U21': 'Switzerland', 'SO Cholet': 'France', 'FC Barnsley': 'England', 'Platanias': 'Not Found', 'FC Münsingen': 'Switzerland', 'Team Vaud U16': 'Not Found', 'Atalanta U19': 'Italy', 'SD Huesca': 'Spain', 'FC St. Pauli': 'Germany', 'Hibernian FC': 'Scotland', 'CD Alcoyano': 'Spain', 'FC Paradiso': 'Switzerland', 'ASEC Mimosas': 'Not Found', 'CD Teneriffa': 'Spain', 'H. Kfar Saba': 'Israel', 'FK Ventspils': 'Latvia', 'Panathinaikos': 'Greece', 'Haugesund': 'Norway', 'AC Mailand': 'Italy', 'Avaí FC': 'Brazil', 'Ismaily': 'Egypt', 'ES Zarzis': 'Not Found', 'Kabuscorp SC': 'Not Found', 'Alanyaspor': 'Turkey', 'pausiert': 'Not Found', 'Cracovia ': 'Poland', 'AS St.-Étienne': 'France', 'Al-Batin': 'Saudi Arabia', 'Galatasaray': 'Turkey', 'Rio Ave FC': 'Portugal', 'Fenerbahce': 'Turkey', 'Bisceglie': 'Italy', 'Mérida AD': 'Spain', 'G. Bordeaux': 'France', 'Azzurri LS': 'Switzerland', 'Maccabi Haifa': 'Israel', 'B. Banja Luka': 'Bosnia-Herzegovina'}
#Check if all clubs are in dictionary
print(len(dict_countries)==len(all_teams_unique))



In [ ]:
#DO NOT EXECUTE, FILES ALREADY CREATED

#Find teams where country is not found
country_not_found ={}
for (key, value) in dict_countries.items():
   if value == "Not Found":
       country_not_found[key] = value

print(len(country_not_found))
print(country_not_found)

#Write to csv and manually search for countries
#Code has been commented in order to not owerwrite "'country_not_found"-file that has been manually adjusted
#with open('country_not_found.csv', 'w', encoding = 'utf-8') as csv_file:  
    #writer = csv.writer(csv_file)
    #for key, value in dict_countries.items():
        #writer.writerow([key, value])




### Create Dataframe

In [17]:
#Add information for clubs that had "Country" == Not Found to dict_countries

#Import adjusted "country-not-found-file"
countries_not_found_df = pd.read_excel('country_not_found.xlsx', index_col=None)

#Replace "Not Found" values with countries from manually adjusted 'country_not_found"-file
for i in (countries_not_found_df.to_dict('split')["data"]):
          dict_countries[i[0]]=i[1]

#Unique set of countries to check for errors
set(dict_countries.values())

#nan value detected, How to handle those?
dict_countries["Karriereende"]="Karriereende"
dict_countries["Unbekannt"]="Unbekannt"



In [ ]:
#For every country find Language Family and Language (manual task)
#set(dict_countries.values())

df_dict_countries = pd.DataFrame.from_dict(dict_countries,orient='index')
df_dict_countries.to_csv("add_language.csv", encoding="utf-8-sig")


#Import excel containing all languages and language families as dataframe

df_countries_languages1= pd.read_csv('added_languages.csv')

df_countries_languages1.head()


#Swiss Teams are divided into French, Italian and German Speaking, therefore seperate mapping necessary
#dict_swiss_team ={}
#for key,value in dict_countries.items():
#    if value =="Switzerland":
#        dict_swiss_team[key]="Switzerland"
#print(dict_swiss_team)

#df_swiss_team = pd.DataFrame.from_dict(dict_swiss_team,orient='index')
#df_swiss_team.to_csv("swiss_teams_lamguage.csv", encoding="utf-8-sig")

#with open('swiss_teams_lamguage.csv', 'w', encoding = 'utf-8') as csv_file:  
#    writer = csv.writer(csv_file)
#    for key,value in dict_swiss_team.items():
#        writer.writerow([key,value])


#Add Language and LanguageFamily for Swiss clubs

#Import Data frame containining information about swiss clubs

df_swiss_teams_language = pd.read_excel('swiss_teams_lamguage.xlsx', index_col=None)
df_swiss_teams_language.rename(columns = {'Country': 'CHCountry', 'Language':'CHLanguage','LanguageFamily':'CHLanguageFamily'}, inplace =True)


df_countries_languages2 = df_countries_languages1.merge(df_swiss_teams_language, how="left",left_on = "Team", right_on = "Team")
#df_countries_languages2.head()

df_countries_languages2.loc[df_countries_languages2.Country == "Switzerland","Language"]=df_countries_languages2["CHLanguage"]
df_countries_languages2.loc[df_countries_languages2.Country == "Switzerland","LanguageFamily"]=df_countries_languages2["CHLanguageFamily"]
#df_countries_languages2.head()

df_language_complete = df_countries_languages2.iloc[:, 0:4]
df_language_complete.head()





In [19]:
#Merge tables
#import all transfers table
df_all_transfers = pd.read_csv("all")


#create six new fields FromCountry and ToCountry, FromLanguage and ToLanguage, FromLanguageFamily and ToLanguageFamilymap countries to clubs

from_added_1 = df_all_transfers.merge(df_language_complete, how="left",left_on = "From", right_on = "Team")
from_added_2 = from_added_1.rename(columns = {'Country': 'FromCountry', 'Language':'FromLanguage','LanguageFamily':'FromLanguageFamily'})
#from_added_2.head()

to_country_added_1 = from_added_2.merge(df_language_complete, how="left",left_on = "To", right_on = "Team")
to_country_added_2 = to_country_added_1.rename(columns = {'Country': 'ToCountry', 'Language':'ToLanguage','LanguageFamily':'ToLanguageFamily'})
#to_country_added_2.head()


df_complete = to_country_added_2[["Season","Name","Transfer Sum","From","FromCountry","FromLanguage","FromLanguageFamily","To","ToCountry","ToLanguage","ToLanguageFamily"]]
df_complete.head()


df_complete.to_csv("final.csv", encoding="utf-8-sig")













### Clean Dataframe

In [ ]:
#We assumed that the clubnames are unique
#During the analysis it was detected that certain clubs on transfermarkt.de have multiple names, the names have to be standardized
#Fc Basel 1893 -> Fc Basel
#Fc St. Gallen 1879 -> Fc St. Gallen
#NE Xamax -> Xamax
#Neuchâtel Xamax FCS -> Xamax

df_complete_multinames = pd.read_csv("final.csv")

df_complete_multinames.info()

In [ ]:


df_complete_uniquenames = df_complete_multinames.replace({"FC Basel 1893":"FC Basel","FC St. Gallen 1879":"FC St. Gallen","NE Xamax":"Xamax","Neuchâtel Xamax FCS":"Xamax"})

df_complete_uniquenames.info()



In [22]:

#If a transfer between 2 Swiss First League clubs occured, the transfer occurs twice in the dataset
#once as Arrival from a club and once as a Departure. In order to correct this we will change the season value
#by removing the string "Abgänge" and "Zugänge" from the "Season" column.

df_complete_uniquenames['Season'] = df_complete_uniquenames['Season'].str.extract(r'(\d{2}/\d{2})', expand=False)

In [ ]:
#Find and remove duplicates
#We check for all columns except the first one which is a unique ID
columnnames = df_complete_uniquenames.columns.tolist()[1:]
columnnames

In [24]:
#Removes duplicates
df_complete_deduped = df_complete_uniquenames.drop_duplicates(subset=columnnames, keep='first')
df_complete_deduped

#pd.unique(df_complete_deduped.duplicated(subset=columnnames, keep='first'))

,Unnamed: 0,Season,Name,Transfer Sum,From,FromCountry,FromLanguage,FromLanguageFamily,To,ToCountry,ToLanguage,ToLanguageFamily
0,0,20/21,Arthur Cabral,"4,40 Mio. €",Palmeiras,Brazil,Portuguese,Romance,FC Basel,Switzerland,German,Germanic
1,1,20/21,Kaly Sene,"4,00 Mio. €",Juventus U19,Italy,Italian,Romance,FC Basel,Switzerland,German,Germanic
2,2,20/21,Edon Zhegrova,"3,00 Mio. €",KRC Genk,Belgium,French,Romance,FC Basel,Switzerland,German,Germanic
3,3,20/21,Andrea Padula,450 Tsd. €,FC Wil 1900,Switzerland,German,Germanic,FC Basel,Switzerland,German,Germanic
4,4,20/21,Heinz Lindner,ablösefrei,Wehen Wiesbaden,Germany,German,Germanic,FC Basel,Switzerland,German,Germanic
...,...,...,...,...,...,...,...,...,...,...,...,...
4366,4366,11/12,Felipe Martins,?,FC Lugano,Switzerland,Italian,Romance,Montreal Impact,United States,English,Germanic
4367,4367,11/12,Patrice Feussi,-,FC Lugano,Switzerland,Italian,Romance,Vereinslos,Vereinslos,Vereinslos,Vereinslos
4368,4368,11/12,Palmiro Di Dio,-,FC Lugano,Switzerland,Italian,Romance,FC Lugano U21,Switzerland,Italian,Romance
4369,4369,11/12,Rijat Shala,-,FC Lugano,Switzerland,Italian,Romance,Vereinslos,Vereinslos,Vereinslos,Vereinslos


In [116]:
df_complete_deduped.to_csv("final_clean.csv", encoding="utf-8-sig")

In [25]:
df_complete_deduped.to_excel("final_clean.xlsx")